## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint-Pierre,RE,-21.3393,55.4781,78.48,82,75,10.36,broken clouds
1,1,Jamestown,US,42.0970,-79.2353,51.53,72,100,8.05,overcast clouds
2,2,Touros,BR,-5.1989,-35.4608,85.69,63,63,13.11,broken clouds
3,3,Kannur,IN,11.8675,75.3576,81.59,84,31,3.27,scattered clouds
4,4,Hithadhoo,MV,-0.6000,73.0833,82.72,61,7,2.55,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint-Pierre,RE,-21.3393,55.4781,78.48,82,75,10.36,broken clouds
6,6,Nanakuli,US,21.3906,-158.1547,76.10,68,7,19.71,clear sky
10,10,Jalu,LY,29.0331,21.5482,76.87,30,0,21.83,clear sky
15,15,Kapaa,US,22.0752,-159.3190,76.19,78,75,16.11,broken clouds
19,19,Isangel,VU,-19.5500,169.2667,79.90,76,100,13.04,overcast clouds
21,21,Carnarvon,AU,-24.8667,113.6333,77.07,78,40,12.66,scattered clouds
24,24,Rikitea,PF,-23.1203,-134.9692,79.57,64,14,15.01,few clouds
26,26,Bengkulu,ID,-3.8004,102.2655,74.14,87,88,4.25,overcast clouds
28,28,Bambous Virieux,MU,-20.3428,57.7575,79.05,83,40,9.22,light rain
31,31,Castro,BR,-24.7911,-50.0119,73.36,71,100,2.15,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


Unnamed: 0             154
City                   154
Country                151
Lat                    154
Lng                    154
Max Temp               154
Humidity               154
Cloudiness             154
Wind Speed             154
Current Description    154
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df=preferred_cities_df.dropna()
preferred_cities_clean_df.count()

Unnamed: 0             151
City                   151
Country                151
Lat                    151
Lng                    151
Max Temp               151
Humidity               151
Cloudiness             151
Wind Speed             151
Current Description    151
dtype: int64

In [7]:
preferred_cities_clean_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Saint-Pierre,RE,-21.3393,55.4781,78.48,82,75,10.36,broken clouds
6,6,Nanakuli,US,21.3906,-158.1547,76.10,68,7,19.71,clear sky
10,10,Jalu,LY,29.0331,21.5482,76.87,30,0,21.83,clear sky
15,15,Kapaa,US,22.0752,-159.3190,76.19,78,75,16.11,broken clouds
19,19,Isangel,VU,-19.5500,169.2667,79.90,76,100,13.04,overcast clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Saint-Pierre,RE,78.48,broken clouds,-21.3393,55.4781,
6,Nanakuli,US,76.10,clear sky,21.3906,-158.1547,
10,Jalu,LY,76.87,clear sky,29.0331,21.5482,
15,Kapaa,US,76.19,broken clouds,22.0752,-159.3190,
19,Isangel,VU,79.90,overcast clouds,-19.5500,169.2667,
21,Carnarvon,AU,77.07,scattered clouds,-24.8667,113.6333,
24,Rikitea,PF,79.57,few clouds,-23.1203,-134.9692,
26,Bengkulu,ID,74.14,overcast clouds,-3.8004,102.2655,
28,Bambous Virieux,MU,79.05,light rain,-20.3428,57.7575,
31,Castro,BR,73.36,overcast clouds,-24.7911,-50.0119,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

hotel_df.head()
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Saint-Pierre,RE,78.48,broken clouds,-21.3393,55.4781,Lindsey Hôtel
6,Nanakuli,US,76.10,clear sky,21.3906,-158.1547,Camp Pālehua
10,Jalu,LY,76.87,clear sky,29.0331,21.5482,صاحبة الفخامة
15,Kapaa,US,76.19,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
19,Isangel,VU,79.90,overcast clouds,-19.5500,169.2667,Tanna Lodge


In [10]:
hotel_df.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 151 entries, 0 to 673
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 151 non-null    object 
 1   Country              151 non-null    object 
 2   Max Temp             151 non-null    float64
 3   Current Description  151 non-null    object 
 4   Lat                  151 non-null    float64
 5   Lng                  151 non-null    float64
 6   Hotel Name           151 non-null    object 
dtypes: float64(3), object(4)
memory usage: 13.5+ KB


In [12]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np

hotel_df['Hotel Name'].replace('', np.nan, inplace=True)

In [13]:
clean_hotel_df=hotel_df.dropna()

In [14]:
clean_hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140 entries, 0 to 673
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 140 non-null    object 
 1   Country              140 non-null    object 
 2   Max Temp             140 non-null    float64
 3   Current Description  140 non-null    object 
 4   Lat                  140 non-null    float64
 5   Lng                  140 non-null    float64
 6   Hotel Name           140 non-null    object 
dtypes: float64(3), object(4)
memory usage: 8.8+ KB


In [15]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Saint-Pierre,RE,78.48,broken clouds,-21.3393,55.4781,Lindsey Hôtel
6,Nanakuli,US,76.10,clear sky,21.3906,-158.1547,Camp Pālehua
10,Jalu,LY,76.87,clear sky,29.0331,21.5482,صاحبة الفخامة
15,Kapaa,US,76.19,broken clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
19,Isangel,VU,79.90,overcast clouds,-19.5500,169.2667,Tanna Lodge
21,Carnarvon,AU,77.07,scattered clouds,-24.8667,113.6333,Hospitality Carnarvon
24,Rikitea,PF,79.57,few clouds,-23.1203,-134.9692,People ThankYou
26,Bengkulu,ID,74.14,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
28,Bambous Virieux,MU,79.05,light rain,-20.3428,57.7575,Casa Tia Villa
31,Castro,BR,73.36,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY


In [16]:
clean_hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140 entries, 0 to 673
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 140 non-null    object 
 1   Country              140 non-null    object 
 2   Max Temp             140 non-null    float64
 3   Current Description  140 non-null    object 
 4   Lat                  140 non-null    float64
 5   Lng                  140 non-null    float64
 6   Hotel Name           140 non-null    object 
dtypes: float64(3), object(4)
memory usage: 8.8+ KB


In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file)



In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 

max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             #max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))